In [1]:
from src.Data import Data
from src.Camera import Camera
from src.VisualOdometry import VisualOdometry
from src.data_association import *
from src.utils import *

import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go   

In [2]:
data = Data()
camera = Camera()

In [3]:
kernel_threshold = 10000
dumping_factor = 2
min_inliners = 8
num_iterations = 15

In [4]:
# def triangulate_points(points_0, points_1, w_T_c0, w_T_c1):

#     K = camera.get_camera_matrix()

#     T = np.linalg.inv(w_T_c1) @ w_T_c0
#     R = T[:3, :3] 
#     t = T[:3, 3].reshape(-1, 1)   

#     #** Projection matrices
#     P_0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
#     P_1 = K @ np.hstack((R, t))        

#     #** Triangulate points
#     points_4D = cv2.triangulatePoints(P_0, P_1, points_0.T, points_1.T)
#     points_4D = w_T_c0 @ points_4D
#     points_3D = points_4D[:3] / points_4D[3]

#     return points_3D.T

def triangulate_point(p1, p2, P1, P2):
    """
    Triangulate a 3D point from two views.

    Parameters:
    p1 (array): The 2D projected point in the first image [x1, y1].
    p2 (array): The 2D projected point in the second image [x2, y2].
    P1 (array): The 3x4 camera projection matrix for the first image.
    P2 (array): The 3x4 camera projection matrix for the second image.

    Returns:
    array: The 3D coordinates of the triangulated point.
    """

    A = np.zeros((4, 4))
    A[0] = p1[0] * P1[2] - P1[0]
    A[1] = p1[1] * P1[2] - P1[1]
    A[2] = p2[0] * P2[2] - P2[0]
    A[3] = p2[1] * P2[2] - P2[1]

    U, S, Vt = np.linalg.svd(A)
    X = Vt[-1]  
    
    return X[:-1] / X[-1]

def triangulate_points(points_1, points_2, pose_1, pose_2):
    """
    Triangulate 3D points from two sets of 2D points.

    Parameters:
    points_1 (array): The 2D projected points in the first image.
    points_2 (array): The 2D projected points in the second image.
    T_0 (array): The 3x4 camera projection matrix for the first image.
    T_1 (array): The 3x4 camera projection matrix for the second image.

    Returns:
    array: The 3D coordinates of the triangulated points.
    """

    K = camera.get_camera_matrix()
    
    points_3d = []
    for i in range(len(points_1)):
        points_1[i] = points_1[i]
        points_2[i] = points_2[i]

        T = np.linalg.inv(pose_2) @ pose_1
        R = T[:3, :3]
        t = T[:3, 3].reshape(-1, 1)
        P_0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
        P_1 = K @ np.hstack((R, t))

        point_3d = triangulate_point(points_1[i], points_2[i], P_0, P_1)
        if point_3d[2] > 0: points_3d.append(point_3d)

    return np.array(points_3d)

def projective_ICP(current_pose, image_points, map):

    for i in range(num_iterations):
        matches = data_association_on_appearance(image_points, map)
        reference_image_points = np.array(matches['points_1'])
        current_world_points = np.array(matches['points_2'])

        w_T_c1, stop = one_step(current_pose, reference_image_points, current_world_points)
        if w_T_c1 is None: return current_pose

        current_pose = w_T_c1
        camera.set_c_T_w(np.linalg.inv(current_pose))

        if stop: break

    return current_pose

def one_step(current_pose, reference_image_points, current_world_points):

        if (len(current_world_points) == 0): return current_pose, True

        H, b, num_inliers, chi_inliers, chi_outliers, error = linearize(current_pose, reference_image_points, current_world_points)
        if num_inliers < min_inliners: return None, None

        H += np.eye(6) * dumping_factor
        dx = np.linalg.solve(H, -b)

        initial_guess = T2v(current_pose)
        updated_guess = initial_guess + dx
        w_T_c1 = v2T(updated_guess)

        return w_T_c1, error<0.1 or np.linalg.norm(dx)<1e-5

def linearize(current_pose, reference_image_points, currrent_world_points):
    H = np.zeros((6, 6))
    b = np.zeros(6)
    num_inliers = 0
    chi_inliers = 0
    chi_outliers = 0

    error = 0

    for i in range(len(reference_image_points)):
        reference_image_point = reference_image_points[i]
        current_world_point = currrent_world_points[i]

        e, jacobian = error_and_jacobian(current_pose, reference_image_point, current_world_point)
        if e is None or jacobian is None: continue

        chi = e.T @ e 
        
        lambda_ = 1.0
        is_inlier = True
        if chi > kernel_threshold:
            lambda_ = np.sqrt(kernel_threshold / chi)
            is_inlier = False
            chi_outliers += chi
        else:
            num_inliers += 1
            chi_inliers += chi

        if is_inlier:
            H += jacobian.T @ jacobian * lambda_
            b += jacobian.T @ e * lambda_

        error += np.linalg.norm(e)
    
    error /= len(reference_image_points)

    return H, b, num_inliers, chi_inliers, chi_outliers, error

def error_and_jacobian(current_pose, reference_image_point, current_world_point):

    is_inside, projected_image_point = camera.project_point(current_world_point)    
    if not is_inside: return None, None

    e = reference_image_point - projected_image_point

    p_hat_hom = np.linalg.inv(current_pose) @ np.append(current_world_point, 1)
    p_hat = p_hat_hom[:3] / p_hat_hom[3]
    p_hat_cam = camera.get_camera_matrix() @ p_hat

    J_icp = np.zeros((3, 6))
    J_icp[:3, :3] = np.eye(3)
    J_icp[:3, 3:] = -skew(p_hat)
    
    z_inv = 1.0 / p_hat_cam[2]
    z_inv2 = z_inv * z_inv

    J_proj = np.array([[z_inv, 0, -p_hat_cam[0] * z_inv2],
                        [0, z_inv, -p_hat_cam[1] * z_inv2]])
    
    J = J_proj @ camera.get_camera_matrix() @ J_icp

    return e, J

In [5]:
poses = []
map = {'position':[], 'appearance':[]}

In [6]:
meas_0 = data.get_measurements_data_points(0)
meas_1 = data.get_measurements_data_points(1)

matches = data_association_on_appearance(meas_0, meas_1)
points_0 = np.array(matches['points_1'])
points_1 = np.array(matches['points_2'])

w_T_c0 = np.eye(4)
poses.append(w_T_c0)

#** Estimate the relative pose between the two frames
K = camera.get_camera_matrix()
E, mask = cv2.findEssentialMat(points_0, points_1, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
_, R, t, mask = cv2.recoverPose(E, points_0, points_1, K, mask=mask)
c0_T_c1 = Rt2T(R, -t)
next_pose = w_T_c0 @ c0_T_c1

print("Estimated relative pose: ")
print(np.round(next_pose, 2))

#** Triangulate points
points_3D = triangulate_points(points_0, points_1, w_T_c0, next_pose)
next_map = {'position':points_3D, 'appearance':matches['appearance']}
# append to map if the points are not already in the map
for i in range(len(next_map['position'])):
    position = next_map['position'][i]
    appearance = next_map['appearance'][i]
    if appearance not in map['appearance']:
        map['position'].append(position)
        map['appearance'].append(appearance)

current_pose = next_pose
current_map = next_map
camera.set_c_T_w(np.linalg.inv(current_pose))
poses.append(current_pose)

#** Plot the 3D points
fig = go.Figure()
x_coords = points_3D[:, 0]  
y_coords = points_3D[:, 1]
z_coords = points_3D[:, 2]
fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=2)))
fig.show()  

Estimated relative pose: 
[[ 1.  0.  0.  0.]
 [-0.  1.  0.  0.]
 [-0. -0.  1.  1.]
 [ 0.  0.  0.  1.]]


In [7]:
for i in range(2,22):
    prev_meas = data.get_measurements_data_points(i-1)
    curr_meas = data.get_measurements_data_points(i)

    next_pose = projective_ICP(current_pose, curr_meas, map)
    print("Estimated relative pose: ")
    print(np.round(next_pose, 2))

    matches = data_association_on_appearance(prev_meas, curr_meas)
    points_1 = np.array(matches['points_1'])
    points_2 = np.array(matches['points_2'])
    points_3D = triangulate_points(points_1, points_2, current_pose, next_pose)
    next_map = {'position':points_3D, 'appearance':matches['appearance']}
    for i in range(len(next_map['position'])):
        position = next_map['position'][i]
        appearance = next_map['appearance'][i]
        if appearance not in map['appearance']:
            map['position'].append(position)
            map['appearance'].append(appearance)
    
    current_pose = next_pose
    current_map = next_map
    camera.set_c_T_w(np.linalg.inv(current_pose))
    poses.append(current_pose)

Estimated relative pose: 
[[ 1.  0.  0.  0.]
 [-0.  1.  0.  0.]
 [-0. -0.  1.  2.]
 [ 0.  0.  0.  1.]]
Estimated relative pose: 
[[ 1.   -0.    0.   -0.02]
 [ 0.    1.   -0.    0.02]
 [-0.    0.    1.    2.99]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.    0.   -0.04]
 [ 0.    1.   -0.    0.05]
 [-0.    0.    1.    4.  ]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.    0.01 -0.08]
 [ 0.    1.   -0.01  0.18]
 [-0.01  0.01  1.    4.95]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.    0.   -0.09]
 [ 0.    1.   -0.02  0.28]
 [-0.    0.02  1.    5.92]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.    0.   -0.18]
 [ 0.    1.   -0.02  0.32]
 [-0.    0.02  1.    6.88]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.   -0.01 -0.  ]
 [ 0.    1.   -0.05  0.57]
 [ 0.01  0.05  1.    7.84]
 [ 0.    0.    0.    1.  ]]
Estimated relative pose: 
[[ 1.   -0.01 -0.01 -0.22]
 [ 0.01  1.   -0.04  0.54]
 [ 0.01

# ROTAZIONE SBAGLIATA?

In [8]:
x_est = []
y_est = []
z_est = []

x_gt = []
y_gt = []
z_gt = []

gt_poses = data.get_trajectory_data()
for i in range(len(poses)):
    C = camera.get_camera_transform()
    pose_i = poses[i]
    pose_i = C @ pose_i
    t_est = pose_i[:3, 3]
    x_est.append(t_est[0]*0.20)
    y_est.append(t_est[1]*0.20)
    z_est.append(t_est[2]*0.20)

    x_gt.append(gt_poses[i][0])
    y_gt.append(gt_poses[i][1])
    z_gt.append(0)

print("num of estimated poses: ", len(x_est))
print("num of ground truth poses: ", len(x_gt))

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_est, y=y_est, z=z_est, mode='lines+markers', marker=dict(size=2, color='red')))
fig.add_trace(go.Scatter3d(x=x_gt, y=y_gt, z=z_gt, mode='lines+markers', marker=dict(size=2, color='green')))
fig.show()

num of estimated poses:  22
num of ground truth poses:  22
